In [2]:
"""
WOA7001 Network Routing Algorithm Implementation - Comparative Version
Filename: network_routing_comparative.py

This version ensures both algorithms output comparable metrics for fair comparison
"""

import heapq
import random
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Set
from collections import defaultdict, deque
import time
import numpy as np

# ==================== Data Structure Definitions ====================

@dataclass
class NetworkNode:
    """Network Node Structure"""
    node_id: str
    name: str
    connected_nodes: List[str]
    traffic_load: str
    link_quality: str

    def __post_init__(self):
        self.connections = set(self.connected_nodes)

@dataclass
class EdgeInfo:
    """Edge Information Structure"""
    quality: float        # Link quality value
    traffic_factor: float # Traffic load factor
    cost: float           # Comprehensive cost

class NetworkGraph:
    """Network Graph Representation"""

    def __init__(self):
        self.nodes: Dict[str, NetworkNode] = {}
        self.edges: Dict[Tuple[str, str], EdgeInfo] = {}

    def add_node(self, node: NetworkNode) -> None:
        """Add node to graph"""
        self.nodes[node.node_id] = node

    def build_edges(self) -> None:
        """Construct bidirectional edges"""
        for node_id, node in self.nodes.items():
            for neighbor_id in node.connections:
                if neighbor_id in self.nodes:
                    self._create_edge(node_id, neighbor_id)

    def _create_edge(self, node1: str, node2: str) -> None:
        """Create edge between two nodes"""
        node1_obj = self.nodes[node1]
        node2_obj = self.nodes[node2]

        # Map link quality to numerical values
        quality_map = {'Excellent': 1.0, 'Good': 2.0, 'Fair': 3.0, 'Poor': 4.0}
        quality1 = quality_map.get(node1_obj.link_quality, 2.5)
        quality2 = quality_map.get(node2_obj.link_quality, 2.5)
        avg_quality = (quality1 + quality2) / 2

        # Map traffic load to numerical factors
        traffic_map = {'High': 2.0, 'Medium': 1.5, 'Low': 1.0}
        traffic1 = traffic_map.get(node1_obj.traffic_load, 1.5)
        traffic2 = traffic_map.get(node2_obj.traffic_load, 1.5)
        avg_traffic = (traffic1 + traffic2) / 2

        # Calculate comprehensive cost (same as A* algorithm)
        cost = avg_quality * avg_traffic

        edge_info = EdgeInfo(
            quality=avg_quality,
            traffic_factor=avg_traffic,
            cost=cost
        )

        # Store bidirectional edges
        self.edges[(node1, node2)] = edge_info
        self.edges[(node2, node1)] = edge_info

    def get_edge(self, node1: str, node2: str) -> Optional[EdgeInfo]:
        """Retrieve edge information"""
        return self.edges.get((node1, node2))

# ==================== Algorithm 1: Multi-Factor A* ====================

class MultiFactorAStarRouter:
    """Multi-Factor A* Routing Algorithm"""

    def __init__(self, graph: NetworkGraph):
        self.graph = graph
        self.performance_stats = {
            'execution_times': [],
            'paths_found': 0,
            'total_operations': 0
        }

    def heuristic(self, node: str, target: str) -> float:
        """
        Heuristic function for A* algorithm
        Returns conservative estimate of cost from node to target
        """
        return 2.0

    def find_path(self, start: str, end: str,
                  quality_weight: float = 0.7,
                  traffic_weight: float = 0.3) -> Tuple[List[str], float, Dict]:
        """
        Find path using Multi-Factor A* algorithm

        Returns:
            (path, total_cost, performance_stats)
        """
        start_time = time.perf_counter()

        if start not in self.graph.nodes or end not in self.graph.nodes:
            return [], float('inf'), {}

        # Initialize A* data structures
        open_set = []
        heapq.heappush(open_set, (0, start))

        came_from = {}

        g_score = {node_id: float('inf') for node_id in self.graph.nodes}
        g_score[start] = 0

        f_score = {node_id: float('inf') for node_id in self.graph.nodes}
        f_score[start] = self.heuristic(start, end)

        nodes_explored = 0

        while open_set:
            _, current = heapq.heappop(open_set)
            nodes_explored += 1

            if current == end:
                # Reconstruct path
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.append(start)
                path.reverse()

                execution_time = (time.perf_counter() - start_time) * 1000

                # Update performance statistics
                self.performance_stats['execution_times'].append(execution_time)
                self.performance_stats['paths_found'] += 1
                self.performance_stats['total_operations'] += nodes_explored

                # Calculate path cost using same method as load balancing
                path_cost = self._calculate_path_cost(path)
                path_metrics = self._calculate_path_metrics(path)

                return path, path_cost, {
                    'execution_time_ms': execution_time,
                    'nodes_explored': nodes_explored,
                    'path_length': len(path) - 1,
                    'path_cost': path_cost,
                    'metrics': path_metrics
                }

            current_node = self.graph.nodes[current]

            for neighbor in current_node.connections:
                if neighbor not in self.graph.nodes:
                    continue

                edge = self.graph.get_edge(current, neighbor)
                if not edge:
                    continue

                # Calculate edge cost (same as load balancing algorithm)
                edge_cost = edge.cost

                tentative_g_score = g_score[current] + edge_cost

                if tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, end)

                    in_open_set = any(neighbor == item[1] for item in open_set)
                    if not in_open_set:
                        heapq.heappush(open_set, (f_score[neighbor], neighbor))

        # No path found
        execution_time = (time.perf_counter() - start_time) * 1000
        return [], float('inf'), {
            'execution_time_ms': execution_time,
            'nodes_explored': nodes_explored,
            'path_length': 0,
            'path_cost': float('inf'),
            'metrics': {}
        }

    def _calculate_path_cost(self, path: List[str]) -> float:
        """Calculate total cost of a path (same method as load balancing)"""
        if len(path) < 2:
            return 0

        total_cost = 0
        for i in range(len(path) - 1):
            edge = self.graph.get_edge(path[i], path[i+1])
            if edge:
                total_cost += edge.cost

        return total_cost

    def _calculate_path_metrics(self, path: List[str]) -> Dict:
        """Calculate various metrics for a path"""
        if len(path) < 2:
            return {}

        total_quality = 0
        total_traffic = 0
        total_cost = 0
        hops = len(path) - 1

        for i in range(hops):
            edge = self.graph.get_edge(path[i], path[i+1])
            if edge:
                total_quality += edge.quality
                total_traffic += edge.traffic_factor
                total_cost += edge.cost

        return {
            'avg_quality': total_quality / hops,
            'avg_traffic': total_traffic / hops,
            'avg_cost': total_cost / hops,
            'hops': hops
        }

    def get_performance_summary(self) -> Dict:
        """Get performance statistics summary"""
        times = self.performance_stats['execution_times']
        return {
            'total_paths_found': self.performance_stats['paths_found'],
            'avg_execution_time_ms': np.mean(times) if times else 0,
            'min_execution_time_ms': min(times) if times else 0,
            'max_execution_time_ms': max(times) if times else 0,
            'avg_operations_per_search': (self.performance_stats['total_operations'] /
                                         self.performance_stats['paths_found']
                                         if self.performance_stats['paths_found'] > 0 else 0)
        }

# ==================== Algorithm 2: Dynamic Load Balancing - Comparative Version ====================

class DynamicLoadBalancerComparative:
    """
    Dynamic Load Balancing Algorithm - Comparative Version

    Modified to output comparable metrics to A* algorithm:
    - Same test node pairs
    - Same cost calculation method
    - Comparable performance metrics
    """

    def __init__(self, graph: NetworkGraph, k_paths: int = 3):
        self.graph = graph
        self.k_paths = k_paths

        # Traffic monitoring (for dynamic simulation only)
        self.node_traffic = defaultdict(int)
        self.path_usage = defaultdict(int)
        self.path_weights = defaultdict(list)

        # Performance statistics for comparative analysis
        self.comparative_stats = {
            'execution_times': [],
            'paths_found': 0,
            'nodes_explored': [],
            'total_operations': 0
        }

        # Path cache for efficiency
        self.path_cache = {}

    def find_static_best_path(self, start: str, end: str) -> Tuple[List[str], float, Dict]:
        """
        Find the best static path for comparative analysis

        This method provides outputs comparable to A* algorithm:
        - Returns single best path (lowest static cost)
        - Calculates execution time
        - Counts nodes explored

        Returns:
            (best_path, path_cost, performance_stats)
        """
        start_time = time.perf_counter()

        if start not in self.graph.nodes or end not in self.graph.nodes:
            return [], float('inf'), {}

        # Find K candidate paths
        paths, nodes_explored = self._find_k_paths_with_count(start, end)

        if not paths:
            execution_time = (time.perf_counter() - start_time) * 1000
            return [], float('inf'), {
                'execution_time_ms': execution_time,
                'nodes_explored': nodes_explored,
                'path_length': 0,
                'path_cost': float('inf')
            }

        # Calculate static cost for each path (same method as A*)
        path_costs = []
        for path in paths:
            cost = self._calculate_static_path_cost(path)
            path_costs.append(cost)

        # Select path with minimum static cost
        best_idx = path_costs.index(min(path_costs))
        best_path = paths[best_idx]
        best_cost = path_costs[best_idx]

        execution_time = (time.perf_counter() - start_time) * 1000

        # Update comparative statistics
        self.comparative_stats['execution_times'].append(execution_time)
        self.comparative_stats['paths_found'] += 1
        self.comparative_stats['nodes_explored'].append(nodes_explored)
        self.comparative_stats['total_operations'] += nodes_explored

        # Calculate path metrics
        path_metrics = self._calculate_path_metrics(best_path)

        return best_path, best_cost, {
            'execution_time_ms': execution_time,
            'nodes_explored': nodes_explored,
            'path_length': len(best_path) - 1,
            'path_cost': best_cost,
            'alternative_paths_found': len(paths),
            'path_index': best_idx,
            'metrics': path_metrics
        }

    def _find_k_paths_with_count(self, start: str, end: str) -> Tuple[List[List[str]], int]:
        """
        Find K candidate paths with node exploration counting

        Returns:
            (list_of_paths, total_nodes_explored)
        """
        cache_key = (start, end, self.k_paths)
        if cache_key in self.path_cache:
            return self.path_cache[cache_key]['paths'], 0

        paths = []
        nodes_explored = 0
        visited_nodes = set()

        # Try different strategies to find multiple paths
        strategies = [
            self._find_path_dijkstra_like,
            self._find_path_avoiding_quality,
            self._find_path_random
        ]

        for strategy in strategies:
            if len(paths) >= self.k_paths:
                break

            path = strategy(start, end, visited_nodes)
            if path and path not in paths:
                paths.append(path)
                # Add nodes to explored count
                for node in path:
                    if node not in visited_nodes:
                        visited_nodes.add(node)
                        nodes_explored += 1

        # If still need more paths, try DFS
        while len(paths) < self.k_paths:
            path = self._find_path_dfs(start, end, visited_nodes, max_depth=4)
            if path and path not in paths:
                paths.append(path)
                for node in path:
                    if node not in visited_nodes:
                        visited_nodes.add(node)
                        nodes_explored += 1
            else:
                break

        self.path_cache[cache_key] = {
            'paths': paths,
            'nodes_explored': nodes_explored
        }

        return paths, nodes_explored

    def _find_path_dijkstra_like(self, start: str, end: str, visited: Set[str]) -> Optional[List[str]]:
        """Find path using Dijkstra-like approach (quality priority)"""
        distances = {node: float('inf') for node in self.graph.nodes}
        previous = {node: None for node in self.graph.nodes}
        distances[start] = 0

        pq = [(0, start)]

        while pq:
            current_dist, current = heapq.heappop(pq)

            if current == end:
                # Reconstruct path
                path = []
                while current is not None:
                    path.append(current)
                    current = previous[current]
                path.reverse()
                return path

            if current_dist > distances[current]:
                continue

            for neighbor in self.graph.nodes[current].connections:
                edge = self.graph.get_edge(current, neighbor)
                if not edge:
                    continue

                new_dist = current_dist + edge.quality  # Prioritize quality

                if new_dist < distances[neighbor]:
                    distances[neighbor] = new_dist
                    previous[neighbor] = current
                    heapq.heappush(pq, (new_dist, neighbor))

        return None

    def _find_path_avoiding_quality(self, start: str, end: str, visited: Set[str]) -> Optional[List[str]]:
        """Find path that may avoid high-quality (expensive) links"""
        distances = {node: float('inf') for node in self.graph.nodes}
        previous = {node: None for node in self.graph.nodes}
        distances[start] = 0

        pq = [(0, start)]

        while pq:
            current_dist, current = heapq.heappop(pq)

            if current == end:
                # Reconstruct path
                path = []
                while current is not None:
                    path.append(current)
                    current = previous[current]
                path.reverse()
                return path

            if current_dist > distances[current]:
                continue

            for neighbor in self.graph.nodes[current].connections:
                edge = self.graph.get_edge(current, neighbor)
                if not edge:
                    continue

                # Invert quality: lower quality gets lower cost
                quality_inverted_cost = 5.0 - edge.quality  # 5.0 is max possible quality
                new_dist = current_dist + quality_inverted_cost

                if new_dist < distances[neighbor]:
                    distances[neighbor] = new_dist
                    previous[neighbor] = current
                    heapq.heappush(pq, (new_dist, neighbor))

        return None

    def _find_path_random(self, start: str, end: str, visited: Set[str]) -> Optional[List[str]]:
        """Find random path for diversity"""
        for attempt in range(20):
            path = [start]
            current = start
            local_visited = {start}

            for step in range(10):  # Limit path length
                if current == end:
                    return path

                # Get unvisited neighbors
                neighbors = [n for n in self.graph.nodes[current].connections
                           if n not in local_visited]

                if not neighbors:
                    break

                # Random selection
                next_node = random.choice(neighbors)
                path.append(next_node)
                local_visited.add(next_node)
                current = next_node

        return None

    def _find_path_dfs(self, start: str, end: str, visited: Set[str], max_depth: int = 4) -> Optional[List[str]]:
        """Find path using Depth-First Search"""

        def dfs(current: str, path: List[str], depth: int) -> Optional[List[str]]:
            if current == end:
                return path.copy()

            if depth >= max_depth:
                return None

            # Shuffle neighbors for random exploration
            neighbors = list(self.graph.nodes[current].connections)
            random.shuffle(neighbors)

            for neighbor in neighbors:
                if neighbor in path:  # Avoid cycles
                    continue

                path.append(neighbor)
                result = dfs(neighbor, path, depth + 1)
                if result:
                    return result
                path.pop()

            return None

        return dfs(start, [start], 0)

    def _calculate_static_path_cost(self, path: List[str]) -> float:
        """Calculate static cost of a path (same method as A* algorithm)"""
        if len(path) < 2:
            return 0

        total_cost = 0
        for i in range(len(path) - 1):
            edge = self.graph.get_edge(path[i], path[i+1])
            if edge:
                total_cost += edge.cost

        return total_cost

    def _calculate_path_metrics(self, path: List[str]) -> Dict:
        """Calculate various metrics for a path"""
        if len(path) < 2:
            return {}

        total_quality = 0
        total_traffic = 0
        total_cost = 0
        hops = len(path) - 1

        for i in range(hops):
            edge = self.graph.get_edge(path[i], path[i+1])
            if edge:
                total_quality += edge.quality
                total_traffic += edge.traffic_factor
                total_cost += edge.cost

        return {
            'avg_quality': total_quality / hops,
            'avg_traffic': total_traffic / hops,
            'avg_cost': total_cost / hops,
            'hops': hops
        }

    def simulate_dynamic_routing(self, start: str, end: str, num_packets: int = 20) -> Dict:
        """
        Simulate dynamic routing with traffic

        This is the original load balancing functionality
        kept for demonstration purposes
        """
        print(f"\nDynamic Load Balancing Simulation: {start} -> {end}")
        print(f"Routing {num_packets} packets with traffic awareness")

        # Find candidate paths
        paths, _ = self._find_k_paths_with_count(start, end)

        if not paths:
            return {'success': False, 'reason': 'No paths available'}

        print(f"Found {len(paths)} candidate paths:")
        for i, path in enumerate(paths):
            path_names = [self.graph.nodes[node].name for node in path]
            static_cost = self._calculate_static_path_cost(path)
            print(f"  Path {i+1}: {' → '.join(path_names)} (Static cost: {static_cost:.2f})")

        # Simulate packet routing with load awareness
        path_counts = [0] * len(paths)
        path_loads = [0.0] * len(paths)

        for packet_num in range(num_packets):
            # Calculate current load for each path
            current_loads = []
            for path in paths:
                path_load = 0
                for node in path:
                    # Simple load model: more traffic = higher load
                    node_load = 1.0 + (self.node_traffic[node] * 0.1)
                    path_load += node_load
                current_loads.append(path_load / len(path))

            # Select path with minimum current load
            selected_idx = current_loads.index(min(current_loads))
            selected_path = paths[selected_idx]

            # Update statistics
            path_counts[selected_idx] += 1
            path_loads[selected_idx] = current_loads[selected_idx]

            # Update node traffic (simulate traffic accumulation)
            for node in selected_path:
                self.node_traffic[node] += 1

            # Simulate traffic decay (every 5 packets)
            if packet_num % 5 == 0:
                for node in self.node_traffic:
                    if self.node_traffic[node] > 0:
                        self.node_traffic[node] -= 1

        # Calculate dynamic metrics
        total_packets = sum(path_counts)
        load_balance_score = 0

        if len(paths) > 1 and total_packets > 0:
            # Calculate load balance: 1 - coefficient of variation
            avg_load = np.mean([c for c in path_counts if c > 0])
            if avg_load > 0:
                variance = np.var([c for c in path_counts if c > 0])
                load_balance_score = 1.0 / (1.0 + variance)

        print("\nDynamic Routing Results:")
        print("-" * 40)
        for i in range(len(paths)):
            if path_counts[i] > 0:
                percentage = (path_counts[i] / total_packets) * 100
                path_names = [self.graph.nodes[node].name for node in paths[i]]
                print(f"  Path {i+1}: {path_counts[i]} packets ({percentage:.1f}%)")
                print(f"    Average load: {path_loads[i]:.2f}")

        print(f"\nLoad Balance Score: {load_balance_score:.3f} (1.0 = perfect balance)")

        return {
            'success': True,
            'paths': paths,
            'path_counts': path_counts,
            'path_loads': path_loads,
            'load_balance_score': load_balance_score,
            'total_packets': total_packets
        }

    def get_comparative_stats_summary(self) -> Dict:
        """Get comparative performance statistics"""
        times = self.comparative_stats['execution_times']
        nodes_explored = self.comparative_stats['nodes_explored']

        return {
            'total_paths_found': self.comparative_stats['paths_found'],
            'avg_execution_time_ms': np.mean(times) if times else 0,
            'min_execution_time_ms': min(times) if times else 0,
            'max_execution_time_ms': max(times) if times else 0,
            'avg_nodes_explored': np.mean(nodes_explored) if nodes_explored else 0,
            'avg_operations_per_search': (self.comparative_stats['total_operations'] /
                                         self.comparative_stats['paths_found']
                                         if self.comparative_stats['paths_found'] > 0 else 0)
        }

# ==================== Network Creation Utility ====================

def create_sample_network() -> NetworkGraph:
    """Create sample network based on Table 4"""
    graph = NetworkGraph()

    graph.add_node(NetworkNode(
        node_id='001',
        name='Router-A',
        connected_nodes=['002', '003', '004'],
        traffic_load='High',
        link_quality='Good'
    ))

    graph.add_node(NetworkNode(
        node_id='002',
        name='Router-B',
        connected_nodes=['001', '003'],
        traffic_load='Medium',
        link_quality='Fair'
    ))

    graph.add_node(NetworkNode(
        node_id='003',
        name='Router-C',
        connected_nodes=['001', '002', '004'],
        traffic_load='High',
        link_quality='Excellent'
    ))

    graph.add_node(NetworkNode(
        node_id='004',
        name='Router-D',
        connected_nodes=['001', '003'],
        traffic_load='Low',
        link_quality='Poor'
    ))

    graph.build_edges()
    return graph

# ==================== Comparative Analysis Function ====================

def run_comparative_analysis():
    """
    Run comprehensive comparative analysis of both algorithms

    Uses the same test cases for both algorithms and
    outputs comparable performance metrics
    """
    print("=" * 70)
    print("WOA7001 Network Routing - Algorithm Comparative Analysis")
    print("=" * 70)

    # Create network
    graph = create_sample_network()

    # Define test cases (same for both algorithms)
    test_cases = [
        ('001', '004'),  # Router-A -> Router-D
        ('002', '004'),  # Router-B -> Router-D
        ('001', '003'),  # Router-A -> Router-C
        ('004', '002'),  # Router-D -> Router-B
    ]

    # Initialize results storage
    a_star_results = []
    load_balancing_results = []
    comparison_table = []

    print("\n1. Running Multi-Factor A* Algorithm Tests")
    print("-" * 50)

    # Test Multi-Factor A* algorithm
    a_star_router = MultiFactorAStarRouter(graph)

    for start, end in test_cases:
        start_name = graph.nodes[start].name
        end_name = graph.nodes[end].name

        print(f"\n   Test: {start_name} -> {end_name}")

        # Run A* algorithm
        a_star_path, a_star_cost, a_star_stats = a_star_router.find_path(start, end)

        if a_star_path:
            path_names = [graph.nodes[node].name for node in a_star_path]
            print(f"     Path: {' → '.join(path_names)}")
            print(f"     Cost: {a_star_cost:.2f}")
            print(f"     Execution time: {a_star_stats['execution_time_ms']:.4f} ms")
            print(f"     Nodes explored: {a_star_stats['nodes_explored']}")
            print(f"     Path length: {a_star_stats['path_length']} hops")

            a_star_results.append({
                'test_case': f"{start}->{end}",
                'path': a_star_path,
                'cost': a_star_cost,
                'stats': a_star_stats
            })
        else:
            print(f"     No path found")

    print("\n\n2. Running Dynamic Load Balancing Algorithm Tests")
    print("-" * 50)

    # Test Dynamic Load Balancing algorithm
    load_balancer = DynamicLoadBalancerComparative(graph, k_paths=3)

    for start, end in test_cases:
        start_name = graph.nodes[start].name
        end_name = graph.nodes[end].name

        print(f"\n   Test: {start_name} -> {end_name}")

        # Run Load Balancing algorithm (static best path)
        lb_path, lb_cost, lb_stats = load_balancer.find_static_best_path(start, end)

        if lb_path:
            path_names = [graph.nodes[node].name for node in lb_path]
            print(f"     Best path: {' → '.join(path_names)}")
            print(f"     Cost: {lb_cost:.2f}")
            print(f"     Execution time: {lb_stats['execution_time_ms']:.4f} ms")
            print(f"     Nodes explored: {lb_stats['nodes_explored']}")
            print(f"     Path length: {lb_stats['path_length']} hops")
            print(f"     Alternative paths found: {lb_stats['alternative_paths_found']}")

            load_balancing_results.append({
                'test_case': f"{start}->{end}",
                'path': lb_path,
                'cost': lb_cost,
                'stats': lb_stats
            })
        else:
            print(f"     No path found")

    # Run one dynamic simulation for demonstration
    print("\n\n3. Dynamic Load Balancing Simulation (Demo)")
    print("-" * 50)
    load_balancer.simulate_dynamic_routing('001', '004', num_packets=15)

    # Generate comparative analysis
    print("\n\n4. Algorithm Performance Comparison")
    print("=" * 50)

    print("\nPerformance Metrics Comparison:")
    print("-" * 70)
    print(f"{'Test Case':<15} {'Algorithm':<20} {'Cost':<10} {'Time (ms)':<12} {'Nodes Explored':<15} {'Path Length':<12}")
    print("-" * 70)

    for i, (a_star_result, lb_result) in enumerate(zip(a_star_results, load_balancing_results)):
        test_case = a_star_result['test_case']

        # A* results
        print(f"{test_case:<15} {'Multi-Factor A*':<20} "
              f"{a_star_result['cost']:<10.2f} "
              f"{a_star_result['stats']['execution_time_ms']:<12.4f} "
              f"{a_star_result['stats']['nodes_explored']:<15} "
              f"{a_star_result['stats']['path_length']:<12}")

        # Load Balancing results
        print(f"{'':<15} {'Load Balancing':<20} "
              f"{lb_result['cost']:<10.2f} "
              f"{lb_result['stats']['execution_time_ms']:<12.4f} "
              f"{lb_result['stats']['nodes_explored']:<15} "
              f"{lb_result['stats']['path_length']:<12}")

        print("-" * 70)

        # Store comparison
        comparison_table.append({
            'test_case': test_case,
            'a_star_cost': a_star_result['cost'],
            'a_star_time': a_star_result['stats']['execution_time_ms'],
            'a_star_nodes': a_star_result['stats']['nodes_explored'],
            'a_star_length': a_star_result['stats']['path_length'],
            'lb_cost': lb_result['cost'],
            'lb_time': lb_result['stats']['execution_time_ms'],
            'lb_nodes': lb_result['stats']['nodes_explored'],
            'lb_length': lb_result['stats']['path_length'],
            'alternative_paths': lb_result['stats']['alternative_paths_found']
        })

    # Calculate summary statistics
    print("\nSummary Statistics:")
    print("-" * 70)

    a_star_times = [r['stats']['execution_time_ms'] for r in a_star_results]
    lb_times = [r['stats']['execution_time_ms'] for r in load_balancing_results]

    a_star_costs = [r['cost'] for r in a_star_results]
    lb_costs = [r['cost'] for r in load_balancing_results]

    a_star_nodes = [r['stats']['nodes_explored'] for r in a_star_results]
    lb_nodes = [r['stats']['nodes_explored'] for r in load_balancing_results]

    print(f"Multi-Factor A* Algorithm:")
    print(f"  Average execution time: {np.mean(a_star_times):.6f} ms")
    print(f"  Average path cost: {np.mean(a_star_costs):.2f}")
    print(f"  Average nodes explored: {np.mean(a_star_nodes):.1f}")

    print(f"\nDynamic Load Balancing Algorithm:")
    print(f"  Average execution time: {np.mean(lb_times):.6f} ms")
    print(f"  Average path cost: {np.mean(lb_costs):.2f}")
    print(f"  Average nodes explored: {np.mean(lb_nodes):.1f}")

    print(f"\nComparative Analysis:")
    print(f"  Time difference (A* - LB): {np.mean(a_star_times) - np.mean(lb_times):.6f} ms")
    print(f"  Cost difference (A* - LB): {np.mean(a_star_costs) - np.mean(lb_costs):.2f}")
    # 修复这行代码：从stats字典中获取alternative_paths_found
    print(f"  Average alternative paths found: {np.mean([r['stats']['alternative_paths_found'] for r in load_balancing_results]):.1f}")

    # Algorithm characteristics summary
    print("\n\n5. Algorithm Characteristics Summary")
    print("=" * 50)

    characteristics = [
        ["Characteristic", "Multi-Factor A*", "Dynamic Load Balancing"],
        ["-"*20, "-"*25, "-"*25],
        ["Primary Objective", "Find optimal single path", "Find multiple good paths"],
        ["Output", "Single path with min cost", "K paths + dynamic selection"],
        ["Time Complexity", "O((V+E)logV)", "O(K×(V+E) + monitoring)"],
        ["Space Complexity", "O(V)", "O(K×V + monitoring data)"],
        ["Best for", "Latency-sensitive flows", "High-throughput applications"],
        ["Adaptability", "Static optimization", "Dynamic real-time adaptation"],
        ["Path Diversity", "Single optimal path", "Multiple alternative paths"]
    ]

    for row in characteristics:
        print(f"{row[0]:<20} {row[1]:<25} {row[2]:<25}")

    return {
        'a_star_results': a_star_results,
        'load_balancing_results': load_balancing_results,
        'comparison_table': comparison_table
    }

# ==================== Main Execution ====================

if __name__ == "__main__":
    """
    Main execution block

    Runs comparative analysis of both routing algorithms
    """
    print("WOA7001 - Network Routing Algorithm Comparative Analysis")
    print("Version: Comparative Output Alignment")
    print("=" * 70)

    # Run comparative analysis
    results = run_comparative_analysis()

    print("\n" + "=" * 70)
    print("Comparative Analysis Complete!")
    print("=" * 70)
    print("\nKey Findings:")
    print("1. Both algorithms successfully found paths for all test cases")
    print("2. Algorithms may select different paths based on their optimization criteria")
    print("3. Multi-Factor A* tends to be faster for single-path computation")
    print("4. Load Balancing provides path diversity for dynamic adaptation")
    print("5. Choice depends on application requirements and network conditions")

WOA7001 - Network Routing Algorithm Comparative Analysis
Version: Comparative Output Alignment
WOA7001 Network Routing - Algorithm Comparative Analysis

1. Running Multi-Factor A* Algorithm Tests
--------------------------------------------------

   Test: Router-A -> Router-D
     Path: Router-A → Router-D
     Cost: 4.50
     Execution time: 0.0338 ms
     Nodes explored: 4
     Path length: 1 hops

   Test: Router-B -> Router-D
     Path: Router-B → Router-C → Router-D
     Cost: 7.25
     Execution time: 0.0131 ms
     Nodes explored: 4
     Path length: 2 hops

   Test: Router-A -> Router-C
     Path: Router-A → Router-C
     Cost: 3.00
     Execution time: 0.0082 ms
     Nodes explored: 2
     Path length: 1 hops

   Test: Router-D -> Router-B
     Path: Router-D → Router-C → Router-B
     Cost: 7.25
     Execution time: 0.0098 ms
     Nodes explored: 4
     Path length: 2 hops


2. Running Dynamic Load Balancing Algorithm Tests
--------------------------------------------------
